# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** Muhammad Alif Nasrulloh
- **Email:** alifnasrulloh.jbg@gmail.com
- **ID Dicoding:** malifnasrulloh

## Menentukan Pertanyaan Bisnis

- Produk apa yang memiliki penjualan terbanyak?
- Apakah ada korelasi antara berat produk dan harga produk?
- Metode penjualan apa yang sering dipakai oleh customer?
- Bagaimana tentang analisa penjualan?


## Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import seaborn as sn
import locale
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt



```
# Ini diformat sebagai kode
```

## Data Wrangling

### Gathering Data

ganti locale ke United States untuk keperluan formatting pada currency

In [ ]:
locale.setlocale(locale.LC_ALL,'en_US.UTF-8')

ambil semua data yang diperlukan dari file csv

In [ ]:
customers = pd.read_csv("customers_dataset.csv")
order_items = pd.read_csv("order_items_dataset.csv")
order_payments = pd.read_csv("order_payments_dataset.csv")
orders = pd.read_csv("orders_dataset.csv")
product_translation = pd.read_csv("product_category_name_translation.csv")
products = pd.read_csv("products_dataset.csv")

### Assessing Data

Ubah bahasa pada tabel product kolom product_category_name agar memakai bahasa inggris

In [ ]:
#set translation
products = pd.merge(products, product_translation, how='inner', on='product_category_name')
products.drop(columns="product_category_name", inplace=True)
products.rename(columns={"product_category_name_english":"product_category_name"}, inplace=True)

fungsi untuk mencari item yang banyak terjual

In [ ]:
def getMostSoldItems(product_df: pd.DataFrame, order_items_df:pd.DataFrame):
    df = pd.merge(product_df, order_items_df, how='inner', on='product_id')
    return df.groupby(by=['product_category_name']).product_category_name.count().sort_values().head(10)

fungsi untuk mendapatkan total order, pakai parameter deliveredOnly untuk menfilter agar menghitung yang sudah status 'delivered'

In [ ]:
def getTotalOrder(order_df:pd.DataFrame, deliveredOnly:bool):
    return len(order_df if deliveredOnly == False else order_df[order_df.order_status == 'delivered'])

fungsi untuk mencari total income selama penjualan

In [ ]:
def getTotalIncome(order_df:pd.DataFrame, order_item_df:pd.DataFrame, deliveredOnly:bool):
    order_df = order_df[order_df.order_status == 'delivered'] if deliveredOnly == True else order_df
    #asumsi saya freight cost dibayar oleh pembeli jadi tidak masuk ke pendapatan
    df = pd.merge(order_df, order_item_df, how='inner',on='order_id')
    return locale.currency(df.price.sum(), grouping=True)

fungsi untuk mencari rata rata banyak item yang terjual setiap per hari

In [ ]:
def getAverageSoldItems(order_df:pd.DataFrame):
    return order_df.groupby(by=['order_purchase_timestamp']).order_purchase_timestamp.value_counts().mean()

fungsi untuk distribute nama produk dengan pembayaran apa yang dipakai

In [ ]:
def getProductPaymentDistribute(product_df:pd.DataFrame, order_items_df:pd.DataFrame, order_payments_df:pd.DataFrame):
    df = pd.merge(product_df, order_items_df, how='inner', on='product_id')
    df = pd.merge(df, order_payments_df, how='inner',on='order_id')
    return df.groupby(by=['product_category_name', 'payment_type']).payment_type.count()

fungsi untuk mencari korelasi antara berat suatu produk denga harga produk itu sendiri

In [ ]:
def getCorrelatProduct(product_df:pd.DataFrame, order_items_df:pd.DataFrame):
    df = pd.merge(product_df, order_items_df, how='inner', on='product_id')
    #kategorikan berdasarkan berat produk dengan rata rata harga yang didapat (top 10 barang terberat)
    return df.groupby(by=['product_weight_g']).agg({'price':'mean'}).sort_values(by=['product_weight_g'],ascending=False).head(10).to_dict()


fungsi membuat dataset baru untuk keperluan rfm nanti

In [ ]:
def createRFM(order_df:pd.DataFrame, order_items_df:pd.DataFrame):
    df = pd.merge(order_df, order_items_df, how='inner',on='order_id')
    df = df.groupby('customer_id_x').agg({
    'order_purchase_timestamp_x': lambda x: (max(order_df['order_purchase_timestamp'])+pd.DateOffset(1) - x.max()).days,
    'order_id': 'count',
    'price': 'sum'
    }).reset_index()
    df.columns = ['customer_id', 'recency', 'frequency', 'monetary']
    print(df)
    return df

### Cleaning Data

In [ ]:
#fungsi untuk drop semua duplicate
def clean_data(df: pd.DataFrame):
    return df.drop_duplicates()

In [ ]:
customers = clean_data(customers)
order_items = clean_data(order_items)
order_payments = clean_data(order_payments)
orders = clean_data(orders)
product_translation = clean_data(product_translation)
products = clean_data(products)

## Exploratory Data Analysis (EDA)

### Explore ...

ubah kolom index ke 3 dan seterusnya menjadi dtype datetime

In [ ]:
#ubah kolom ke tipe datetime
for i in orders.columns.tolist()[3:]:
    orders[i] = pd.to_datetime(orders[i])

In [ ]:
#untuk mendapatkan tanggal paling awal dan akhir di semua penjualan
first_date_order = orders.order_purchase_timestamp.min()
last_date_order = orders.order_purchase_timestamp.max()

In [ ]:
#untuk memilih range tanggal data yang mau ditampilkan
with st.sidebar:
    first_date, last_date = st.date_input(label='Plese select date range', value=[first_date_order, last_date_order], max_value=last_date_order, min_value=first_date_order)
    search = st.text_input("Check Order ID")
    with st.expander("Result: "):
        st.write(orders.loc[search == orders.order_id])

In [ ]:
#filter untuk mneyaring agar hanya tampil sesuai range tanggal
filtered_orders = orders[(orders["order_purchase_timestamp"] >= str(first_date)) &  (orders["order_purchase_timestamp"] <= str(last_date))]
filtered_orders_items = pd.merge(order_items, filtered_orders, how='inner', on='order_id')

## Visualization & Explanatory Analysis

In [ ]:
st.header("E-Commerce Report")



```
# Ini diformat sebagai kode
```

### Pertanyaan 1:

In [ ]:
st.subheader("Analisa Produk")
columns = st.columns(2, gap='medium')
with columns[0]:
    st.text("Top 10 Frekuensi Product E-Commerce")

    fig, ax = plt.subplots()
    product_frequent = products.groupby(by=["product_category_name"]).product_category_name.count().to_dict()
    product_frequent = dict(sorted(product_frequent.items(),key=lambda x: x[1], reverse=True)[:7])
    plt.pie(x = product_frequent.values(), labels=product_frequent.keys(), autopct='%1.1f%%', explode=list(map(lambda x: 0.2 if x == max(product_frequent.values()) else 0, product_frequent.values())))

    st.pyplot(fig)

with columns[1]:
    st.text("Produk dengan Penjualan Terbanyak")

    fig, ax = plt.subplots()
    mostSoldItem = getMostSoldItems(product_df=products, order_items_df=filtered_orders_items).to_dict()
    plt.barh(y=list(mostSoldItem.keys()), width=list(mostSoldItem.values()))
    st.pyplot(fig)



```
# Ini diformat sebagai kode
```

### Pertanyaan 2:

In [ ]:
st.text("Korelasi Berat Produk dengan Harga Produk")
fig, ax = plt.subplots()
data = getCorrelatProduct(products, order_items)['price']
sn.scatterplot(x=data.keys(), y=data.values(), )
st.pyplot(fig)



```
# Ini diformat sebagai kode
```

### Pertanyaan 3:

In [ ]:
st.subheader("Analisa Metode Pembayaran")

payDistribute = getProductPaymentDistribute(products,order_items,order_payments).to_dict()
method_payment = ['boleto', 'credit_card', 'debit_card', 'voucher']

temp = {}
for k,v in payDistribute.items():
    if k[0] in mostSoldItem.keys():
        if k[0] not in temp.keys():
            temp[k[0]] = {}
        temp[k[0]][k[1]]=v

#fill another method with 0
for k,v in temp.items():
    for i in method_payment:
        if i not in v.keys():
            temp[k][i] = 0

barWidth = 0.25
fig,ax = plt.subplots()

boleto = list(map(lambda x: x['boleto'],temp.values()))
credit_card = list(map(lambda x: x['credit_card'],temp.values()))
debit_card = list(map(lambda x: x['debit_card'],temp.values()))
voucher = list(map(lambda x: x['voucher'],temp.values()))

br1 = np.arange(len(boleto))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]

plt.bar(br1, boleto, color ='r', width = barWidth,
        edgecolor ='grey', label ='Boleto')
plt.bar(br2, credit_card, color ='g', width = barWidth,
        edgecolor ='grey', label ='Credit Card')
plt.bar(br3, debit_card, color ='b', width = barWidth,
        edgecolor ='grey', label ='Debit Card')
plt.bar(br4, voucher, color ='b', width = barWidth,
        edgecolor ='grey', label ='Voucher')

plt.xlabel('Product Name', fontweight ='bold', fontsize = 15)
plt.ylabel('Banyak Pemakaian', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(boleto))],
        mostSoldItem.keys(), rotation=75)
plt.legend()

st.pyplot(fig)




```
# Ini diformat sebagai kode
```

### Pertanyaan 4:

In [ ]:
t.subheader("Analisa Penjualan")
columns = st.columns(3, gap='medium')
with columns[0]:
    st.metric(label="Total Penjualan", value=getTotalOrder(filtered_orders, False))
    st.metric(label="Total Penjualan (Delivered Only)", value=getTotalOrder(filtered_orders, True))

with columns[1]:
    st.metric(label="Total Pendapatan", value=getTotalIncome(filtered_orders, order_items, False))
    st.metric(label="Total Pendapatan (Delivered Only)", value=getTotalIncome(filtered_orders, order_items, True))

with columns[2]:
    st.metric(label="Rata-Rata Barang Terjual Per-Hari", value=round(getAverageSoldItems(filtered_orders)))

## Conclusion

- 10 Produk penjualan terbanyak ditampilkan pada gambar
- Ada
- Credit Card
- Lengkapnya sudah ditampilkan di dashboard

# RFM Analysis



In [ ]:
st.subheader("Best RFM Analysis")

rfm_df = createRFM(filtered_orders, filtered_orders_items)

col1, col2, col3 = st.columns(3)

with col1:
    avg_recency = round(rfm_df.recency.mean(), 1)
    st.metric("Rata-Rata Ketepatan Waktu Pembelian", value=avg_recency)
    fig,ax = plt.subplot(1, 3, 1)
    plt.hist(rfm_df['recency'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    plt.title('Recency Distribution')
    plt.xlabel('Recency (days)')
    plt.ylabel('Number of Customers')
    st.pyplot(fig)

with col2:
    avg_frequency = round(rfm_df.frequency.mean(), 2)
    st.metric("Rata-Rata Frekuensi Pembelian", value=avg_frequency)
    fig,ax = plt.subplot(1, 3, 2)
    plt.hist(rfm_df['frequency'], bins=20, color='salmon', edgecolor='black', alpha=0.7)
    plt.title('Frequency Distribution')
    plt.xlabel('Frequency')
    plt.ylabel('Number of Customers')
    st.pyplot(fig)

with col3:
    avg_moneter = locale.currency(rfm_df.monetary.mean(),grouping=True)
    st.metric("Rata-Rata Moneter Pembelian", value=avg_moneter)
    fig,ax = plt.subplot(1, 3, 3)
    plt.hist(rfm_df['monetary'], bins=20, color='lightgreen', edgecolor='black', alpha=0.7)
    plt.title('Monetary Distribution')
    plt.xlabel('Monetary')
    plt.ylabel('Number of Customers')
    st.pyplot(fig)
